In [2]:
import pandas as pd
restaurants_df = pd.read_csv('Restaurants.csv')
license_df = pd.read_csv('Liquor_Licenses.csv')
restaurants_df.rename(columns = {'X':'Latitude', 'Y':'Longitude'}, inplace = True)
license_df.rename(columns = {'street_address':'Address', 'dba':'Name', 'X':'Latitude', 'Y':'Longitude'}, inplace = True)

In [3]:
all_df = pd.merge(restaurants_df, license_df, how='outer', on='Address')
all_df = all_df.drop_duplicates(
  subset = ['Name_x', 'Address'],
  keep = 'last').reset_index(drop = True)
all_df = all_df[~all_df['Name_x'].isna()] #get rid of places that are not restaurants
all_df[['Name_x', 'Name_y', 'Address']]

# all_df.loc[all_df['number'].isna()==True, 'Serve_Liquor'] = 'No' #create new column, number stands for liquor license number
# all_df.loc[all_df['number'].isna()==False, 'Serve_Liquor'] = 'Yes'

    

,Name_x,Name_y,Address
0,1509 Populace LLC,NaN,1509 Broadway St
1,Albena- Taste Everything LLC,NaN,1509 Broadway St
2,HMWW Food LLC dba Karl's The Siren,NaN,1509 Broadway St
3,The Wurlitzer MT DBA Candy Bar,NaN,1509 Broadway St
4,1917 American Bistro,NaN,19416 Livernois
5,1st Wok,NaN,12555 Grand River Ave #150
6,234 Winder St. Inn Community Center,234 WINDER STREET INN,2458 Brush St
7,Brush St. Stadium Deli,234 WINDER STREET INN,2458 Brush St
8,24 Grille,NaN,204 Michigan Ave
9,2941 Street Foods of Midtown,2941 STREET FOOD,4219 Woodward Ave


In [34]:
all_df[all_df['Serve_Liquor'] == 'Yes'] #there are 412 restaurants who serve liquor
all_df[all_df['Serve_Liquor'] == 'No'] #there are 1940 restaurants who don't...
all_df

,Latitude_x,Longitude_x,Establishment_ID,Name_x,Address,Zip_Code,Owner,License_Number,License_Type,Most_Recent_License_Date,...,group_,type,subtype,statute,number,status,ownership_transferable,location_transferable,ObjectId_y,Serve_Liquor
0,-83.048647,42.335696,51622592.0,1509 Populace LLC,1509 Broadway St,48226,Populace Coffee/Andrew Heppner,226550,Fixed Establishment,2019/04/27 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,-83.048647,42.335696,51622594.0,Albena- Taste Everything LLC,1509 Broadway St,48226,Garrett Lipar,229298,Fixed Establishment,2019/04/26 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,-83.048647,42.335696,51622842.0,HMWW Food LLC dba Karl's The Siren,1509 Broadway St,48226,Matt Wang,238151,Fixed Establishment,2019/04/16 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,-83.048647,42.335696,51622595.0,The Wurlitzer MT DBA Candy Bar,1509 Broadway St,48226,HM Ventures 6,229302,Fixed Establishment,2019/04/16 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,-83.141277,42.435563,3928.0,1917 American Bistro,19416 Livernois,48221,Donald Studvent,065464,Fixed Establishment,2013/04/30 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
5,-83.167389,42.379641,3153.0,1st Wok,12555 Grand River Ave #150,48204,Ang Lam,047622,Fixed Establishment,1970/01/01 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
6,-83.048753,42.342411,5248.0,234 Winder St. Inn Community Center,2458 Brush St,48201,Mona Ross/ Gardner,217906,Fixed Establishment,2018/05/07 00:00:00+00,...,Retail - Off Premises,Specially Designated Merchant,NaN,MCL 436.1533(5)(a),L-000434839,Escrow,Y,N,1085.0,Yes
7,-83.048753,42.342411,51623047.0,Brush St. Stadium Deli,2458 Brush St,48201,Mona Ross Gardner,239989,Fixed Establishment,2020/04/03 00:00:00+00,...,Retail - Off Premises,Specially Designated Merchant,NaN,MCL 436.1533(5)(a),L-000434839,Escrow,Y,N,1085.0,Yes
8,-83.048428,42.331710,3622.0,24 Grille,204 Michigan Ave,48226,Bistro 24 LLC,064219,Fixed Establishment,2019/04/29 00:00:00+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
9,-83.061432,42.351782,5276.0,2941 Street Foods of Midtown,4219 Woodward Ave,48201,Jacques Van Staden,208218,Fixed Establishment,2019/06/12 00:00:00+00,...,Retail - On Premises,Class C,Regular,NaN,L-000272282,Escrow,Y,Y,1209.0,Yes


In [1]:
!pip install numpy==1.6.0

     |████████████████████████████████| 2.6MB 27.8MB/s eta 0:00:01
    ERROR: Command errored out with exit status 1:
     command: /opt/conda/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-2sf9ahps/numpy/setup.py'"'"'; __file__='"'"'/tmp/pip-install-2sf9ahps/numpy/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: /tmp/pip-install-2sf9ahps/numpy/
    Complete output (755 lines):
    RefactoringTool: Skipping optional fixer: buffer
    RefactoringTool: Skipping optional fixer: idioms
    RefactoringTool: Skipping optional fixer: set_literal
    RefactoringTool: Skipping optional fixer: ws_comma
    RefactoringTool: Refactored /tmp/pip-install-2sf9ahps/numpy/build/py3k/numpy/__init__.py
    RefactoringTool: Refactored /tmp/pip-install-2sf9ahps/numpy/build/py3k/numpy/_import_tools.p